In [87]:
# File Name: SageCheckIWnkViaILPAgainstSignedPermutationsDate29082022Ver01.txt
# Purpose: Check (Integer) Weighing matrices Via Integer Linear Programming against (direct) Signed permutations
# Remark: After session with Radi. We are left with a question:
#         Why No solution for R1,R2:  [0, 0, 0, 0, 0, 0, 5] [0, 0, 2, 2, 2, 2, 3]
# Improvements: (1) Problem from remark fixed.
#               (2) Added "sors" routine from Assaf, so to generalize the whole activity.
# Remark: Radi on 8/5/22 noticed that solutions begin with negative values although BegWithPos should prevent it.
#         We fix it.
# Remark: On 12/5/22 we added two routines
#         OurMain2: Given parameters of weighing matrix, weight and 2 divisors of order, it gives all pairs of
#                   orthogonal rows of all types available of nsoks and filter them by the code invariany method.
#         AddLine:  Given Nsoks and Sols gives all additions to Sols by one row taken from Nsoks
#                   with permutations and signings and codeinvariant.
#                   This routine is the basis for recursion to construct all Integer Weighing Matrices.
#                   We have not run and debugged this routine yet. Next time...
# Remark: On 15/5/22 we wroteqimproved/touched some routines:
#         OurMain2: This is the new main. It gets the [WMweight, WMOrderDiv] of a weighing matrix and should output all inequivalent Integer weighing matrices of order=WMOrderDiv & weight=sqrt(WMweight. 
#         FilterOrthogonalSignedPermutationsAgainstMatrix (haven't debugged yet)
#         CodePrevRows (debugged carefully)
# Remark: On 26052022 we debuged OurMain2 routine.
# Remark On 30-6-22 Organizing the hp check. 
#                   automatic base calculation for the code invariant
#                   indexing the permutation and signs and saving the index in the configuration.
# Remark on 3-7-22 Adding code to check for Hadamrd equivalnce for two rows for permutation AND row negation
#                  Changes in "FilterOrthogonalSignedPermutationsByCodeInvariant" adding two code invariants for each line.
# Remark on 14-7-22 Adding the index for each line of (NsoksIndex,PermutationIndex,SignsIndex) 
#                    Bug for line [3, 2, 2, 2, 2, 0, 0], [[1, 16, 5], (2, -3, 2, -2, 0, 2, 0)]], 
#                                 The SignsIndex looks incorrect - TBD in the next meeting
# Remark on 21-7-22 Generelizing for AboveMatrix.
#                    -   OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                    -   normAboveAgainstMatrix(AboveMatrix,R2):
#                    -   dpAgainstMatrix
#                       need to correct FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
# Remark on 24-7-22  began adapting FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                    adding routine genCode for generating a code for the column i.
#                   for next time - continue to adapt  FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                                    and remove CodePrevRows
# Remark on 31-7-22 remove CodePrevTows and AddLine
#                   Checked FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix for addition of one row.
#                   Coded OurMain3 to work with FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                   Checked OurMain3 for two lines.
# Remark on 31-7-22 Adapt OurMain3 for 7 lines
#                   While running OurMain3 it works fine for 2 and 3 lines but crashes while
#                             calculating for 4 lines in memory shortage
# Remark on 18-7-22 Cleanup
# Remark on 28-8-22 
#                  integrate   FilterOrthogonalSignedPermutationsByCodeInvariant
#                           to OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate hp into OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate gencode also
#                  add goodPerm test to eliminate (-1) assignment to 0 (faster)


def HadamardSpace(d):
    SpaceSize = 2^d
    L=[ZZ(n).digits(2) for n in range(SpaceSize)]
    L1 = [x+[0]*(d-len(x)) for x in L]
    return([vector([2*y-1 for y in x]) for x in L1])

def IsBegWithPos(L):
    for x in L:
        if x>0:
            return(True)
        if x<0:
            return(False)
    return(False)

def goodPerm(L,s):
    for i in range(len(L)):
        if (L[i]==0) and (s[i]==-1) : return false
    return true

def normAboveAgainstMatrix(AboveMatrix,R2):
    for i in range(len(R2)):
        if AboveMatrix.column(i).norm()==0:
            R2[i]=abs(R2[i])
    return R2


def dpAgainstMatrix(AboveMatrix,L2):
    return((AboveMatrix*vector(L2)).norm())


def OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,L,Lindex,MaxV):
    P = Permutations(L).list()
    d = len(L)
    S = HadamardSpace(d)
    SP = []
    s = []
    sd = [] 
    codeBase = 1+2*MaxV
    nR=AboveMat.nrows()
    cB=(vector([codeBase^(nR-j) for j in range(nR)]))*AboveMat
    for pi in range(Lindex[1],len(P)):
        p = P[pi]
        s0=Lindex[2] if pi==Lindex[1] else 0
        for si in range(s0,len(S)):
            sn=S[si]
            if (goodPerm(p,sn)):
                r =[p[i]*sn[i] for i in range(d)]
                if IsBegWithPos(r):
                    if dpAgainstMatrix(AboveMat,r)==0:
                        r=normAboveAgainstMatrix(AboveMat,r)
                        if not(r in SP):
                            SP.append(r)
                            R2i=vector(r)
                            M1 = cB+R2i
                            M1s = sorted(M1)
                            if not(M1s in s) :
                                M1m = cB-R2i
                                M1sm = sorted(M1m)
                                if not(M1sm in s):
                                    s.append(M1s)
                                    sd.append([[Lindex[0],pi,si],R2i])                    
    return(sd)



def sors(n,r,maxsq):  ### Find all ways to express n=\sum_i s_i^2 as a sum of r squares, with maxsq>=s_1>=s_2>=...>=s_r.
    #n0=RDF(n)
    n0=n
    if r==1:
        s=sqrt(n0)
        fs=floor(s)
        if s==fs and s<=maxsq:
            return [[s]]
        else:
            return false
    else:
        sors2=[]
        for x in range(floor(sqrt(n0/r)),min(floor(sqrt(n0)),maxsq)+1):
            m=n-x^2
            #print x
            sors1=sors(m,r-1,x)
            if sors1:
                sors2+=[[x]+sors1[i] for i in range(len(sors1))   ]
           # print "sors2=",sors2
        if len(sors2)>0:
            return sors2
        else:
            return false


def OurMain3(WMWeight, WMOrderDiv,lines):
    MaxSquare = WMWeight^(1/2)
    AllSoks = sors(WMWeight, WMOrderDiv, MaxSquare)
    print("Nsoks: ",AllSoks)
    Sols= [[matrix(1,WMOrderDiv,AllSoks[i]),[i,0,0]] for i in range(len(AllSoks))]
    print(Sols)
    for line in range(lines-1):
        nSols=[]
        countSol=0
        countScan=0
        for Sol in Sols:
            countScan+=1
            AboveMat=Sol[0]
            R1Index=Sol[1]
            for R2i in range(R1Index[0],len(AllSoks)):
                R2Index=R1Index if R2i==R1Index[0] else [R2i,0,0]
                R2 = vector(AllSoks[R2i])
                cpF = OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,R2,R2Index,MaxSquare)
                countSol+=len(cpF)
                if len(cpF)>0:
                    nSols.extend([AboveMat,x] for x in cpF)
            print("countScan,countSol  = ",countScan,countSol)
        Sols=[]
        for nSol in nSols:
            NL=matrix(WMOrderDiv,1,list(nSol[1][1]))
            AM=((nSol[0].transpose()).augment(NL)).transpose()
            Sols.append([AM,nSol[1][0]])
        print("# of sols in line",line, "  =",len(Sols),[Sols[i] for i in range(min(len(Sols),3))])
    return(Sols)

from gc import collect
collect()
%time res=OurMain3(8,8,8)

Nsoks:  [[1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0]]
[[[1 1 1 1 1 1 1 1], [0, 0, 0]], [[2 1 1 1 1 0 0 0], [1, 0, 0]], [[2 2 0 0 0 0 0 0], [2, 0, 0]]]
countScan,countSol  =  1 3
countScan,countSol  =  2 13
countScan,countSol  =  3 15
# of sols in line 0   = 15 [[[ 1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1 -1 -1 -1], [0, 0, 15]], [[ 1  1  1  1  1  1  1  1]
[ 2  1 -1 -1 -1  0  0  0], [1, 0, 227]], [[ 1  1  1  1  1  1  1  1]
[ 2 -2  0  0  0  0  0  0], [2, 0, 253]]]
countScan,countSol  =  1 5
countScan,countSol  =  2 18
countScan,countSol  =  3 19
countScan,countSol  =  4 25
countScan,countSol  =  5 50
countScan,countSol  =  6 62
countScan,countSol  =  7 86
countScan,countSol  =  8 97
countScan,countSol  =  9 108
countScan,countSol  =  10 115
countScan,countSol  =  11 125
countScan,countSol  =  12 127
countScan,countSol  =  13 128
countScan,countSol  =  14 129
countScan,countSol  =  15 131
# of sols in line 1   = 131 [[[ 1  1  1  1  1  1  1  1]
[ 1  1  1  1 -

countScan,countSol  =  91 188
countScan,countSol  =  92 189
countScan,countSol  =  93 189
countScan,countSol  =  94 191
countScan,countSol  =  95 193
countScan,countSol  =  96 195
countScan,countSol  =  97 197
countScan,countSol  =  98 198
countScan,countSol  =  99 199
countScan,countSol  =  100 201
countScan,countSol  =  101 201
countScan,countSol  =  102 201
countScan,countSol  =  103 202
countScan,countSol  =  104 203
countScan,countSol  =  105 203
countScan,countSol  =  106 203
countScan,countSol  =  107 204
countScan,countSol  =  108 205
countScan,countSol  =  109 205
countScan,countSol  =  110 205
countScan,countSol  =  111 206
countScan,countSol  =  112 207
countScan,countSol  =  113 207
countScan,countSol  =  114 208
countScan,countSol  =  115 208
countScan,countSol  =  116 209
countScan,countSol  =  117 209
countScan,countSol  =  118 210
countScan,countSol  =  119 211
countScan,countSol  =  120 212
countScan,countSol  =  121 213
countScan,countSol  =  122 213
countScan,countSo

countScan,countSol  =  356 573
countScan,countSol  =  357 576
countScan,countSol  =  358 576
countScan,countSol  =  359 577
countScan,countSol  =  360 577
countScan,countSol  =  361 577
countScan,countSol  =  362 581
countScan,countSol  =  363 584
countScan,countSol  =  364 587
countScan,countSol  =  365 590
countScan,countSol  =  366 593
countScan,countSol  =  367 595
countScan,countSol  =  368 597
countScan,countSol  =  369 599
countScan,countSol  =  370 601
countScan,countSol  =  371 603
countScan,countSol  =  372 604
countScan,countSol  =  373 605
countScan,countSol  =  374 605
countScan,countSol  =  375 605
countScan,countSol  =  376 609
countScan,countSol  =  377 612
countScan,countSol  =  378 616
countScan,countSol  =  379 619
countScan,countSol  =  380 621
countScan,countSol  =  381 624
countScan,countSol  =  382 628
countScan,countSol  =  383 629
countScan,countSol  =  384 629
countScan,countSol  =  385 630
countScan,countSol  =  386 630
countScan,countSol  =  387 631
countSca

countScan,countSol  =  621 928
countScan,countSol  =  622 929
countScan,countSol  =  623 929
countScan,countSol  =  624 930
countScan,countSol  =  625 930
countScan,countSol  =  626 931
countScan,countSol  =  627 931
countScan,countSol  =  628 932
countScan,countSol  =  629 933
countScan,countSol  =  630 933
countScan,countSol  =  631 933
countScan,countSol  =  632 933
countScan,countSol  =  633 933
countScan,countSol  =  634 933
countScan,countSol  =  635 935
countScan,countSol  =  636 936
countScan,countSol  =  637 937
countScan,countSol  =  638 940
countScan,countSol  =  639 942
countScan,countSol  =  640 943
countScan,countSol  =  641 943
countScan,countSol  =  642 945
countScan,countSol  =  643 946
countScan,countSol  =  644 946
countScan,countSol  =  645 947
countScan,countSol  =  646 947
countScan,countSol  =  647 948
countScan,countSol  =  648 951
countScan,countSol  =  649 952
countScan,countSol  =  650 952
countScan,countSol  =  651 952
countScan,countSol  =  652 952
countSca

countScan,countSol  =  183 163
countScan,countSol  =  184 164
countScan,countSol  =  185 164
countScan,countSol  =  186 165
countScan,countSol  =  187 165
countScan,countSol  =  188 165
countScan,countSol  =  189 165
countScan,countSol  =  190 166
countScan,countSol  =  191 166
countScan,countSol  =  192 167
countScan,countSol  =  193 167
countScan,countSol  =  194 168
countScan,countSol  =  195 168
countScan,countSol  =  196 169
countScan,countSol  =  197 169
countScan,countSol  =  198 169
countScan,countSol  =  199 169
countScan,countSol  =  200 169
countScan,countSol  =  201 169
countScan,countSol  =  202 169
countScan,countSol  =  203 169
countScan,countSol  =  204 169
countScan,countSol  =  205 169
countScan,countSol  =  206 169
countScan,countSol  =  207 169
countScan,countSol  =  208 169
countScan,countSol  =  209 169
countScan,countSol  =  210 169
countScan,countSol  =  211 169
countScan,countSol  =  212 169
countScan,countSol  =  213 169
countScan,countSol  =  214 169
countSca

...

countScan,countSol  =  713 548
countScan,countSol  =  714 548
countScan,countSol  =  715 551
countScan,countSol  =  716 553
countScan,countSol  =  717 554
countScan,countSol  =  718 554
countScan,countSol  =  719 556
countScan,countSol  =  720 557
countScan,countSol  =  721 557
countScan,countSol  =  722 558
countScan,countSol  =  723 558
countScan,countSol  =  724 559
countScan,countSol  =  725 559
countScan,countSol  =  726 560
countScan,countSol  =  727 560
countScan,countSol  =  728 560
countScan,countSol  =  729 560
countScan,countSol  =  730 562
countScan,countSol  =  731 563
countScan,countSol  =  732 563
countScan,countSol  =  733 565
countScan,countSol  =  734 566
countScan,countSol  =  735 566
countScan,countSol  =  736 568
countScan,countSol  =  737 569
countScan,countSol  =  738 569
countScan,countSol  =  739 571
countScan,countSol  =  740 572
countScan,countSol  =  741 572
countScan,countSol  =  742 574
countScan,countSol  =  743 575
countScan,countSol  =  744 575
countSca

countScan,countSol  =  978 703
countScan,countSol  =  979 703
countScan,countSol  =  980 703
countScan,countSol  =  981 703
countScan,countSol  =  982 705
countScan,countSol  =  983 706
countScan,countSol  =  984 707
countScan,countSol  =  985 708
countScan,countSol  =  986 709
countScan,countSol  =  987 710
countScan,countSol  =  988 710
# of sols in line 4   = 710 [[[ 1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1 -1 -1 -1]
[ 1  1 -1 -1  1  1 -1 -1]
[ 1 -1  1 -1  1 -1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1  1 -1 -1  1], [0, 0, 153]], [[ 1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1 -1 -1 -1]
[ 1  1 -1 -1  1  1 -1 -1]
[ 1 -1  1 -1  1 -1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1  1 -1 -1  1 -1  1], [0, 0, 165]], [[ 1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1 -1 -1 -1]
[ 1  1 -1 -1  1  1 -1 -1]
[ 1 -1  1 -1  1 -1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1  1 -1 -1 -1 -1  1  1], [0, 0, 195]]]
countScan,countSol  =  1 2
countScan,countSol  =  2 3
countScan,countSol  =  3 3
countScan,countSol  =  4 4
countS

countScan,countSol  =  248 115
countScan,countSol  =  249 116
countScan,countSol  =  250 116
countScan,countSol  =  251 117
countScan,countSol  =  252 117
countScan,countSol  =  253 118
countScan,countSol  =  254 118
countScan,countSol  =  255 118
countScan,countSol  =  256 119
countScan,countSol  =  257 119
countScan,countSol  =  258 119
countScan,countSol  =  259 119
countScan,countSol  =  260 119
countScan,countSol  =  261 119
countScan,countSol  =  262 120
countScan,countSol  =  263 121
countScan,countSol  =  264 122
countScan,countSol  =  265 122
countScan,countSol  =  266 123
countScan,countSol  =  267 123
countScan,countSol  =  268 124
countScan,countSol  =  269 124
countScan,countSol  =  270 124
countScan,countSol  =  271 124
countScan,countSol  =  272 124
countScan,countSol  =  273 125
countScan,countSol  =  274 126
countScan,countSol  =  275 127
countScan,countSol  =  276 127
countScan,countSol  =  277 128
countScan,countSol  =  278 128
countScan,countSol  =  279 129
countSca

countScan,countSol  =  513 223
countScan,countSol  =  514 224
countScan,countSol  =  515 224
countScan,countSol  =  516 225
countScan,countSol  =  517 225
countScan,countSol  =  518 225
countScan,countSol  =  519 226
countScan,countSol  =  520 226
countScan,countSol  =  521 226
countScan,countSol  =  522 226
countScan,countSol  =  523 226
countScan,countSol  =  524 226
countScan,countSol  =  525 226
countScan,countSol  =  526 226
countScan,countSol  =  527 228
countScan,countSol  =  528 229
countScan,countSol  =  529 229
countScan,countSol  =  530 230
countScan,countSol  =  531 230
countScan,countSol  =  532 230
countScan,countSol  =  533 231
countScan,countSol  =  534 231
countScan,countSol  =  535 231
countScan,countSol  =  536 233
countScan,countSol  =  537 234
countScan,countSol  =  538 234
countScan,countSol  =  539 235
countScan,countSol  =  540 235
countScan,countSol  =  541 235
countScan,countSol  =  542 235
countScan,countSol  =  543 235
countScan,countSol  =  544 235
countSca

countScan,countSol  =  55 11
countScan,countSol  =  56 11
countScan,countSol  =  57 12
countScan,countSol  =  58 12
countScan,countSol  =  59 12
countScan,countSol  =  60 12
countScan,countSol  =  61 12
countScan,countSol  =  62 12
countScan,countSol  =  63 12
countScan,countSol  =  64 12
countScan,countSol  =  65 13
countScan,countSol  =  66 13
countScan,countSol  =  67 13
countScan,countSol  =  68 13
countScan,countSol  =  69 14
countScan,countSol  =  70 14
countScan,countSol  =  71 14
countScan,countSol  =  72 14
countScan,countSol  =  73 14
countScan,countSol  =  74 14
countScan,countSol  =  75 15
countScan,countSol  =  76 15
countScan,countSol  =  77 15
countScan,countSol  =  78 16
countScan,countSol  =  79 16
countScan,countSol  =  80 16
countScan,countSol  =  81 16
countScan,countSol  =  82 16
countScan,countSol  =  83 17
countScan,countSol  =  84 17
countScan,countSol  =  85 17
countScan,countSol  =  86 17
countScan,countSol  =  87 18
countScan,countSol  =  88 18
countScan,coun

In [89]:
save (res,'IW(8,8)')

In [90]:
res = load('IW(8,8)')

In [92]:
res

[[
[ 1  1  1  1  1  1  1  1]             
[ 1  1  1  1 -1 -1 -1 -1]             
[ 1  1 -1 -1  1  1 -1 -1]             
[ 1 -1  1 -1  1 -1  1 -1]             
[ 1 -1 -1  1 -1  1  1 -1]             
[ 1 -1 -1  1  1 -1 -1  1]             
[ 1 -1  1 -1 -1  1 -1  1]             
[ 1  1 -1 -1 -1 -1  1  1], [0, 0, 195]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 2 -1 -1  0 -1  0  0  1]               
[ 0  1  1 -2 -1  0  0  1]               
[ 0  1 -1  0  1 -2  0  1]               
[ 0  1 -1  0 -1  0  2 -1], [1, 225, 107]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 1 -2  0  1  0  1 -1  0]               
[ 1  0 -2  1  0 -1  1  0]               
[ 1  0  0 -1 -2  1  1  0]               
[ 1  0  0 -1  0 -1 -1  2], [1, 170, 151]
],
 [
[ 1  1  1  